# Bokeh example

In [ ]:
import os

from selenium import webdriver
from selenium.webdriver.firefox.service import Service as FirefoxService
from webdriver_manager.firefox import GeckoDriverManager

from bokeh.io import output_file, export_svgs, export_png
import svglib.svglib as svglib
from reportlab.graphics import renderPDF


class FirefoxWebdriver(object):
    def __new__(cls):
        if not hasattr(cls, 'instance'):
            cls.instance = super(FirefoxWebdriver, cls).__new__(cls)

            opts = webdriver.FirefoxOptions()
            opts.add_argument("--headless")
            cls.driver = webdriver.Firefox(service=FirefoxService(GeckoDriverManager().install()), options=opts)
            
        return cls.instance

def export_pdf(bokeh_plot, filename):
    bokeh_plot.output_backend = "svg"
    filename = re.sub(r'\.pdf$','',filename)
    svg_name = filename+'.svg'
    pdf_name = filename+'.pdf'
    
    export_svgs(bokeh_plot, filename=svg_name, webdriver=FirefoxWebdriver().driver)
    svg = svglib.svg2rlg(svg_name)
    renderPDF.drawToFile(svg, pdf_name)
    os.remove(svg_name)

In [ ]:
import math
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource, FactorRange
from bokeh.plotting import figure, show
from bokeh.transform import factor_cmap
output_notebook()

# Code from https://docs.bokeh.org/en/latest/docs/gallery/bar_nested_colormapped.html
fruits = ['Apples', 'Pears', 'Nectarines', 'Plums', 'Grapes', 'Strawberries']
years = ['2015', '2016', '2017']

data = {'fruits' : fruits,
        '2015'   : [2, 1, 4, 3, 2, 4],
        '2016'   : [5, 3, 3, 2, 4, 6],
        '2017'   : [3, 2, 4, 4, 5, 3]}

palette = ["#c9d9d3", "#718dbf", "#e84d60"]

# this creates [ ("Apples", "2015"), ("Apples", "2016"), ("Apples", "2017"), ("Pears", "2015), ... ]
x = [ (fruit, year) for fruit in fruits for year in years ]
counts = sum(zip(data['2015'], data['2016'], data['2017']), ()) # like an hstack

source = ColumnDataSource(data=dict(x=x, counts=counts))

p = figure(x_range=FactorRange(*x), height=350, title="Fruit Counts by Year",
           toolbar_location=None, tools="")

p.vbar(x='x', top='counts', width=0.9, source=source, line_color="white",
       fill_color=factor_cmap('x', palette=palette, factors=years, start=1, end=2))

p.y_range.start = 0
p.x_range.range_padding = 0.1
p.xaxis.major_label_orientation = 1
p.xgrid.grid_line_color = None

show(p)
export_pdf(p, 'plot')